In [1]:
import numpy as np
from torch.utils.data import Dataset,DataLoader
import torch

In [2]:
import pandas as pd

In [3]:
train_data=pd.read_csv("./datasets/mnist_fashion/fashion-mnist_train.csv").values
test_data=pd.read_csv("./datasets/mnist_fashion/fashion-mnist_test.csv").values

In [4]:
class MNISTFashionDataSet(Dataset):
    def __init__(self,dataframe):
        self.data=dataframe
    def __getitem__(self,idx):
        return torch.tensor(self.data[idx,1:]/255,dtype=torch.float32),torch.tensor(self.data[idx,0],dtype=torch.long)
    def __len__(self):
        return self.data.shape[0]

In [5]:
train_data_loader=DataLoader(dataset=MNISTFashionDataSet(dataframe=train_data),shuffle=True,batch_size=100,pin_memory=True)
test_data_loader=DataLoader(dataset=MNISTFashionDataSet(dataframe=test_data),shuffle=False,batch_size=100,pin_memory=True)

In [6]:
from torch.nn import Sequential,Linear,ReLU,Module,CrossEntropyLoss,Dropout,BatchNorm1d
from torch.optim import AdamW

In [23]:
class ClothingClassifier(Module):
    def __init__(self,inputs,outputs):
        super().__init__()
        self.network=Sequential(Linear(784,256),
                                BatchNorm1d(256),
                                ReLU(),
                                Dropout(p=0.3),
                                Linear(256,128),
                                BatchNorm1d(128),
                                ReLU(),
                                Dropout(p=0.3),
                                Linear(128,10))
    def forward(self,inputs):
        return self.network(inputs)

In [24]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

In [25]:
epochs=100
classifier=ClothingClassifier(784,10).to().to(device)
criterion=CrossEntropyLoss()
optimizer=AdamW(params=classifier.parameters(),weight_decay=1e-4)
for epoch in range(epochs):
    epoch_total_loss=0
    epoch_acc=0
    for batch in train_data_loader:
        batch[0]=batch[0].to(device)
        batch[1]=batch[1].to(device)
        y_pred=classifier.forward(batch[0])
        loss=criterion(y_pred,batch[1])
        epoch_acc+=(torch.argmax(y_pred,dim=1)==batch[1]).float().mean()
        epoch_total_loss+=loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"epoch {epoch}\tLoss:{epoch_total_loss/len(train_data_loader)} Acc:{epoch_acc/len(train_data_loader)}")

epoch 64	Loss:0.12463397836933533 Acc:0.9536342620849609
epoch 65	Loss:0.12656917424872519 Acc:0.9531012773513794
epoch 66	Loss:0.12409327171742916 Acc:0.9528512358665466
epoch 67	Loss:0.12075381949543953 Acc:0.9557172656059265
epoch 68	Loss:0.12449865966724853 Acc:0.952850878238678
epoch 69	Loss:0.12282876986389359 Acc:0.9538344144821167
epoch 70	Loss:0.12096916068345308 Acc:0.954684317111969
epoch 71	Loss:0.1194280090307196 Acc:0.9550507664680481
epoch 72	Loss:0.11906139937229454 Acc:0.9549003839492798
epoch 73	Loss:0.11697267076931894 Acc:0.9569345116615295
epoch 74	Loss:0.11527265465197464 Acc:0.9560506343841553
epoch 75	Loss:0.1155059574296077 Acc:0.9569829106330872
epoch 76	Loss:0.11571676112090548 Acc:0.9568341970443726
epoch 77	Loss:0.11682962307706475 Acc:0.9561675786972046
epoch 78	Loss:0.11016222697372238 Acc:0.9587010741233826
epoch 79	Loss:0.11285284116243323 Acc:0.9574839472770691
epoch 80	Loss:0.11084490816108883 Acc:0.9589670896530151
epoch 81	Loss:0.10869173698748151 A

In [28]:
classifier.eval()

ClothingClassifier(
  (network): Sequential(
    (0): Linear(in_features=784, out_features=256, bias=True)
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=256, out_features=128, bias=True)
    (5): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.3, inplace=False)
    (8): Linear(in_features=128, out_features=10, bias=True)
  )
)

In [29]:
acc_sum=0
for data in test_data_loader:
    with torch.no_grad():
        pred=classifier(data[0].to(device))
        acc_sum+=(torch.argmax(pred,dim=1)==(data[1].to(device))).float().mean().item()
print(f"Test Accuracy: {acc_sum/len(test_data_loader)}")

Test Accuracy: 0.9065000039339065


In [7]:
class ClothingClassifier(Module):
    def __init__(self,inputs,outputs,num_hidden_layers,num_neurons_layer):
        super().__init__()
        layers=[Linear(inputs,num_neurons_layer)]
        layers.append(ReLU())
        for i in range(num_hidden_layers):
            layers.append(num_neurons_layer,num_neurons_layer)
            layers.append(ReLU())
        layers.append(Linear(num_neurons_layer,outputs))
        self.network=Sequential(*layers)
        # self.network=Sequential(Linear(784,256),
        #                         BatchNorm1d(256),
        #                         ReLU(),
        #                         Dropout(p=0.3),
        #                         Linear(256,128),
        #                         BatchNorm1d(128),
        #                         ReLU(),
        #                         Dropout(p=0.3),
        #                         Linear(128,10))
    def forward(self,inputs):
        return self.network(inputs)